<a href="https://colab.research.google.com/github/aureavaleria/DataBalancing-Research/blob/main/papers/Artigo%201/V3/Vers%C3%A3o_3_(ajustes_de_hiperparmetros_Gradient_Boosting).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### ***Machine learning for predicting liver and/or lung metastasis in colorectal cancer: A retrospective study based on the SEER database***

Este notebook apresenta um processo de otimização de hiperparâmetros para o modelo Gradient Boosting, utilizando dados extraídos da base SEER, amplamente utilizada em pesquisas oncológicas. O objetivo é ajustar os parâmetros do modelo de forma iterativa, maximizando sua performance com base na métrica AUC-ROC.

### 1. Configuração inicial e importação das bibliotecas

In [1]:
# Importação das bibliotecas essenciais
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.preprocessing import LabelEncoder

### 2. Carregamento e tratamento de dados

In [2]:
# Carregar o dataset
df = pd.read_csv('https://raw.githubusercontent.com/aureavaleria/Reprodu-o/main/export.csv')

# Verificar e remover valores faltantes
print("Valores faltantes por coluna:\n", df.isnull().sum())
df.dropna(inplace=True)

Valores faltantes por coluna:
 Patient ID                                         0
Age recode with <1 year olds                       0
Sex                                                0
Race recode (White, Black, Other)                  0
Histologic Type ICD-O-3                            0
Grade Recode (thru 2017)                           0
Primary Site                                       0
Derived AJCC T, 7th ed (2010-2015)                 0
Derived AJCC N, 7th ed (2010-2015)                 0
CS tumor size (2004-2015)                          0
CEA Pretreatment Interpretation Recode (2010+)     0
Tumor Deposits Recode (2010+)                      0
Marital status at diagnosis                        0
Origin recode NHIA (Hispanic, Non-Hisp)            0
SEER Combined Mets at DX-lung (2010+)             15
SEER Combined Mets at DX-liver (2010+)            12
SEER Combined Mets at DX-bone (2010+)             14
ICD-O-3 Hist/behav                                 0
ICD-O-3 Hist/be

### 3. Divisão de variáveis preditoras (X) e alvo (y)

In [3]:
# Definir as variáveis preditoras (X)
X = df[['Age recode with <1 year olds', 'Sex', 'Race recode (White, Black, Other)',
        'Histologic Type ICD-O-3', 'Grade Recode (thru 2017)', 'Primary Site',
        'Derived AJCC T, 7th ed (2010-2015)', 'Derived AJCC N, 7th ed (2010-2015)',
        'CS tumor size (2004-2015)', 'CEA Pretreatment Interpretation Recode (2010+)',
        'Tumor Deposits Recode (2010+)', 'Marital status at diagnosis',
        'Origin recode NHIA (Hispanic, Non-Hisp)']]

# Definir as variáveis alvo (y)
y = df[['SEER Combined Mets at DX-liver (2010+)', 'SEER Combined Mets at DX-lung (2010+)']]

### 4. Codificação das variáveis categóricas

In [4]:
# Aplicar LabelEncoder às colunas categóricas em X
for col in X.columns:
    if X[col].dtype == 'object':  # Verifica se a coluna é categórica (strings)
        X[col] = LabelEncoder().fit_transform(X[col])

# Criar coluna binária 'Binary Mets'
y['Binary Mets'] = np.where((y['SEER Combined Mets at DX-liver (2010+)'] == 'Yes') |
                            (y['SEER Combined Mets at DX-lung (2010+)'] == 'Yes'), 1, 0)

# Verificar os tamanhos de X e y
print(f"Tamanho de X: {len(X)}")
print(f"Tamanho de y: {len(y)}")

# Renomear colunas de X para remover caracteres inválidos
X.columns = [str(col).replace('[', '').replace(']', '').replace('<', '').replace('>', '').replace(' ', '_')
             for col in X.columns]

Tamanho de X: 53448
Tamanho de y: 53448


<ipython-input-4-47dc951dc1ef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-4-47dc951dc1ef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[col] = LabelEncoder().fit_transform(X[col])
<ipython-input-4-47dc951dc1ef>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

### 5. Otimização de hiperparâmetros

In [5]:
# Função para otimizar hiperparâmetros em grupos
def optimize_hyperparameters(X, y, param_group, fixed_params=None):
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
    model = GradientBoostingClassifier(random_state=42, **(fixed_params or {}))
    grid_search = GridSearchCV(estimator=model, param_grid=param_group, scoring="roc_auc", cv=cv, n_jobs=-1, verbose=1)
    grid_search.fit(X, y)
    return grid_search.best_params_, grid_search.best_score_

# Hiperparâmetros divididos em grupos
param_groups = [
    {"n_estimators": [100, 200], "max_depth": [3, 6]},
    {"learning_rate": [0.01, 0.1], "subsample": [0.8, 1.0]},
    {"min_samples_split": [2, 5], "min_samples_leaf": [1, 5]}
]

# Variáveis fixadas (começa vazio e é atualizado a cada etapa)
fixed_params = {}

# Iterar pelos grupos de hiperparâmetros
for i, param_group in enumerate(param_groups):
    print(f"\nOtimização do grupo {i + 1}/{len(param_groups)}:")
    best_params, best_score = optimize_hyperparameters(X, y['Binary Mets'], param_group, fixed_params)
    fixed_params.update(best_params)
    print(f"Melhores parâmetros no grupo {i + 1}: {best_params}")
    print(f"Pontuação no grupo {i + 1} (AUC-ROC): {best_score:.4f}")

# Parâmetros finais otimizados
print("\nParâmetros finais otimizados:", fixed_params)

# Salvar resultados em arquivo
with open("melhores_hiperparametros_gb_dividido.txt", "w") as file:
    file.write("Modelo: Gradient Boosting (Otimização em Grupos)\n")
    file.write("Parâmetros Finais:\n")
    for param, value in fixed_params.items():
        file.write(f"  {param}: {value}\n")



Otimização do grupo 1/3:
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhores parâmetros no grupo 1: {'max_depth': 3, 'n_estimators': 200}
Pontuação no grupo 1 (AUC-ROC): 0.8855

Otimização do grupo 2/3:
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhores parâmetros no grupo 2: {'learning_rate': 0.1, 'subsample': 0.8}
Pontuação no grupo 2 (AUC-ROC): 0.8858

Otimização do grupo 3/3:
Fitting 5 folds for each of 4 candidates, totalling 20 fits
Melhores parâmetros no grupo 3: {'min_samples_leaf': 5, 'min_samples_split': 2}
Pontuação no grupo 3 (AUC-ROC): 0.8858

Parâmetros finais otimizados: {'max_depth': 3, 'n_estimators': 200, 'learning_rate': 0.1, 'subsample': 0.8, 'min_samples_leaf': 5, 'min_samples_split': 2}
